### Importing the necessary dependencies:

In [148]:
import pandas as pd
import numpy as np

In [149]:
from sklearn.utils import resample
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix,classification_report,roc_auc_score
import statistics
from statistics import mode

### Importing the data files:

#statistical analysis on features:
df=pd.read_csv('/home/vboxuser/VoiceFiles/Voice4PD-MSA-Tool-global_V2/PhonationData/stat_ph/phonationfeaturesst_seg_train_stat.csv')
#train and test data:
df_train=pd.read_csv('/home/vboxuser/VoiceFiles/Voice4PD-MSA-Tool-global_V2/PhonationData/phonationfeaturesst_seg_dev.csv')
df_test=pd.read_csv('/home/vboxuser/VoiceFiles/Voice4PD-MSA-Tool-global_V2/PhonationData/phonationfeaturesst_seg_test.csv')

In [155]:
df=pd.read_csv('/home/vboxuser/VoiceFiles/Voice4PD-MSA-Tool-global_V2/ArticulationData/stat_ar/Articulationfeaturesst_train_stat.csv')

df_train=pd.read_csv('/home/vboxuser/VoiceFiles/Voice4PD-MSA-Tool-global_V2/ArticulationData/articulationfeaturesst_seg_dev.csv')
df_test=pd.read_csv('/home/vboxuser/VoiceFiles/Voice4PD-MSA-Tool-global_V2/ArticulationData/articulationfeaturesst_seg_test.csv')

## Building a model with feature selection:

### Preprocessing the data:

#### 1. Extract the features that are a good discriminant:

In [156]:
df.head()

,feature,p_HC_DP,p_HC_DP.1
0,avg BBEon_1,0.023689,-0.440402
1,avg BBEon_2,0.068999,-0.355201
2,avg BBEon_3,0.027615,-0.381407
3,avg BBEon_4,0.006813,-0.469835
4,avg BBEon_5,0.002928,-0.504544


In [157]:
df_train

,subject,avg BBEon_1,avg BBEon_2,avg BBEon_3,avg BBEon_4,avg BBEon_5,avg BBEon_6,avg BBEon_7,avg BBEon_8,avg BBEon_9,...,kurtosis DDMFCCoff_10,kurtosis DDMFCCoff_11,kurtosis DDMFCCoff_12,kurtosis F1,kurtosis DF1,kurtosis DDF1,kurtosis F2,kurtosis DF2,kurtosis DDF2,Group
0,300_P_0.wav,-1.236051,-1.141281,-1.401868,-2.007595,-2.151657,-2.313118,-2.741410,-3.021620,-3.317513,...,0.204765,-0.634935,-1.341562,1.619932,3.838965,3.611199,-0.823670,6.217124,4.769782,HC
1,300_P_1.wav,-0.094454,-0.378113,-0.758611,-1.256991,-1.342239,-1.376264,-1.561052,-1.530774,-2.029228,...,0.373218,-0.684170,-0.839666,1.249121,3.385462,3.629113,0.910676,5.565311,4.313787,HC
2,300_P_10.wav,-0.279615,-0.118771,-0.317394,-1.148637,-1.395024,-1.330388,-1.775505,-1.994570,-2.286245,...,1.121168,0.033208,-0.894862,1.845131,2.680086,2.076892,0.134500,4.527984,1.688521,HC
3,300_P_11.wav,-0.869392,-0.815159,-1.399940,-1.789387,-2.052317,-2.407308,-2.764290,-2.708282,-3.032211,...,-0.073108,0.283412,-0.546401,3.407841,7.563080,7.010788,-0.012514,3.180005,2.825297,HC
4,300_P_12.wav,-1.470171,-1.281990,-2.004765,-2.429003,-2.444694,-2.423398,-2.645003,-2.804510,-2.778626,...,-0.147155,-0.511000,-0.003748,-0.127163,2.066544,2.093169,-0.439147,1.710982,0.734801,HC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5018,713_P_5.wav,-1.697040,-1.774542,-2.269848,-2.538483,-2.710381,-3.237338,-3.664977,-4.016116,-4.044144,...,0.180870,-0.019783,3.985714,4.483945,11.661424,12.707809,0.331794,5.410056,4.091611,HC
5019,713_P_6.wav,-1.774050,-1.965119,-2.507317,-2.482628,-2.527717,-2.910955,-3.254227,-3.722705,-4.044041,...,3.151396,2.717255,4.476775,3.098299,4.200562,3.905340,0.199770,3.828449,2.729184,HC
5020,713_P_7.wav,-1.679538,-2.044537,-2.523164,-2.826860,-2.692031,-2.838001,-3.564375,-3.916020,-3.884838,...,0.422919,4.375059,-0.159177,0.101693,4.136868,3.815546,-0.588536,2.317244,1.740533,HC
5021,713_P_8.wav,-1.671376,-1.618963,-1.956912,-2.288242,-2.232473,-2.530588,-3.096826,-3.475840,-3.892578,...,-0.822151,0.881160,0.680996,4.045269,7.084508,5.797237,1.364558,15.205907,10.914103,HC


In [158]:
#from the {ImageFolder..} file obtained earlier from feature_analysis we take those values that have p_val<0.05 and effect_size>|0.4|
req_features=df[(df['p_HC_DP']<0.05) & (abs(df['p_HC_DP.1'])>0.4)]

#### 2. Obtain the selected good features as a list:

In [159]:
feature_list=req_features['feature'].tolist()
print(len(feature_list))

139


#### 3. Obtaining X_train & Y_train:

In [160]:
df_train=df_train.fillna(0)

In [161]:
X_train=df_train[df_train.columns.intersection(feature_list)]
X_train.head()

,avg BBEon_1,avg BBEon_4,avg BBEon_5,avg BBEon_6,avg BBEon_7,avg BBEon_8,avg BBEon_9,avg BBEon_10,avg BBEon_11,avg BBEon_12,...,skewness DDMFCCon_5,skewness DDMFCCon_10,skewness BBEoff_6,skewness BBEoff_22,skewness MFCCoff_5,skewness DMFCCoff_12,skewness DDMFCCoff_5,kurtosis BBEon_22,kurtosis BBEoff_2,kurtosis BBEoff_22
0,-1.236051,-2.007595,-2.151657,-2.313118,-2.741410,-3.021620,-3.317513,-3.573329,-3.479685,-3.259063,...,-0.390031,-0.641420,-0.331088,-0.006502,0.004985,-0.081187,-0.487728,-0.815727,-0.681074,-1.390721
1,-0.094454,-1.256991,-1.342239,-1.376264,-1.561052,-1.530774,-2.029228,-2.030290,-1.702845,-1.903537,...,0.168796,0.464600,0.736957,0.949908,0.260764,0.355529,-0.104616,0.807542,-1.264207,-0.079672
2,-0.279615,-1.148637,-1.395024,-1.330388,-1.775505,-1.994570,-2.286245,-2.571636,-2.583259,-2.723721,...,0.790789,-0.613135,-0.710361,0.895773,-0.589876,-0.673822,-0.129467,-0.923050,-0.515471,1.220242
3,-0.869392,-1.789387,-2.052317,-2.407308,-2.764290,-2.708282,-3.032211,-3.328638,-3.416836,-3.325427,...,0.524319,0.313464,0.810267,-0.092846,-1.419273,0.196726,0.469045,0.434434,-1.092410,-1.136694
4,-1.470171,-2.429003,-2.444694,-2.423398,-2.645003,-2.804510,-2.778626,-3.112789,-3.369516,-3.530014,...,-1.154369,-0.171489,-0.430283,-0.602846,0.765616,1.338386,-0.414994,-1.350462,-0.921273,-0.813865


In [162]:
X_train.isnull().sum()

avg BBEon_1             0
avg BBEon_4             0
avg BBEon_5             0
avg BBEon_6             0
avg BBEon_7             0
                       ..
skewness DMFCCoff_12    0
skewness DDMFCCoff_5    0
kurtosis BBEon_22       0
kurtosis BBEoff_2       0
kurtosis BBEoff_22      0
Length: 139, dtype: int64

In [163]:
X_train.shape

(5023, 139)

In [164]:
#replace string values to 0s & 1s in target column
df_train.replace({'Group':{"HC":0,"DP":1}},inplace=True)

In [165]:
Y_train=df_train['Group']
Y_train.head()

0    0
1    0
2    0
3    0
4    0
Name: Group, dtype: int64

#### 4. Resampling the data: 

In [166]:
#we notice an imbalance in the data classes (close to a ratio of 1/3)
val_count=df_train.groupby(['Group'])['Group'].count()
print(val_count)

Group
0    3878
1    1145
Name: Group, dtype: int64


##### 4.1. Upsampling the data

#hence we resample the data to obtain equal number of data points for each of the two classes:
df_1=df_train[df_train['Group']==1]
df_other=df_train[df_train['Group']!=1]
df_upsampled=resample(df_1,random_state=42,n_samples=3878,replace=True)
df_train_upsampled=pd.concat([df_upsampled,df_other])
#df_train_upsampled.reset_index(inplace=True)
#we upsample the minority class to the number of data points available for majority class

In [167]:
#df_train_upsampled=df_train_upsampled.fillna(0)

In [168]:
#X_train_resampled=df_train_upsampled[df_train_upsampled.columns.intersection(feature_list)]
#Y_train_resampled=df_train_upsampled['Group']

In [169]:
#X_train_resampled.head()

In [170]:
#X_train_resampled.isnull().sum()

In [171]:
! pip install imblearn --quiet

In [172]:
from imblearn.over_sampling import SMOTE

sm = SMOTE(sampling_strategy='minority', random_state=42)

X_train_resampled, Y_train_resampled = sm.fit_resample(X_train,Y_train)
df_train_upsampled = pd.concat([pd.DataFrame(Y_train_resampled), pd.DataFrame(X_train_resampled)], axis=1)

In [173]:
#val_count=c.groupby(['Group'])['Group'].count()
#print(val_count)

##### 4.2. Normalizing the data

In [174]:
import Normalization as n

In [175]:
X=n.normalize_train(X_train_resampled)

In [176]:
Y=Y_train_resampled

In [177]:
#checking if normalized data has a mean~0 & standard deviation~1:
print(np.mean(X))
print(np.std(X))

-5.755079538011133e-17
1.0


In [178]:
df_train_upsampled['Group'].value_counts()

0    3878
1    3878
Name: Group, dtype: int64

### Building the model:

#### 1.1  test data:

#### 1. Random Forest Classifier:

In [179]:
#RF_model=RandomForestClassifier(class_weight='balanced')

In [180]:
#for resampled data:
#RF_model.fit(X,Y)

In [181]:
df_test=df_test.fillna(0)

In [182]:
#obtaining X & Y for test data:
df_test.replace({'Group':{"HC":0,"DP":1}},inplace=True)

In [183]:
X_test=df_test[df_test.columns.intersection(feature_list)] 
X_test=n.normalize_test(X_test) 
Y_test=df_test['Group'] 

In [184]:
len(feature_list)

139

import itertools
subject = df_test['subject']
# empty list
result = []

iterator = itertools.groupby(subject, lambda string: string.split('_')[0])

for element, group in iterator:
   
   result.append(list(group))


__FOR SINGLE SPEAKER:__

from collections import Counter

test=df_test[df_test['subject'].isin(result[2])] 
X_test=test[test.columns.intersection(feature_list)]
X_test=n.normalize_test(X_test)
Y_test=test['Group']
Y_predict_RF=RF_model.predict(X_test)

op=Y_predict_RF

p=np.sum(Y_test==1)#/Y_predict_RF.shape[0]*100

p

Y_test.value_counts()

true_per=[]
out_per=[]
true_output=[]
pred_output=[]
count=[]
true_count=[]


for i in range (0,len(result)):
    test=df_test[df_test['subject'].isin(result[i])] 
    X_test=test[test.columns.intersection(feature_list)]
    X_test=n.normalize_test(X_test)
    Y_test=test['Group']

    Y_predict_RF=RF_model.predict(X_test)
    
    per_pred_dp=np.sum(Y_predict_RF==1)/Y_predict_RF.shape[0]*100
    count_pred_dp=np.sum(Y_predict_RF==1)
    count.append(count_pred_dp)
    
    if per_pred_dp>9:
        y_=1
    else:
        y_=0
    
    y=mode(Y_test)
    per=np.sum(Y_test==1)/Y_test.shape[0]*100
    count_true_dp=np.sum(Y_test==1)
    true_count.append(count_true_dp)
    
    
    true_per.append(per)
    out_per.append(per_pred_dp)
    true_output.append(y)
    pred_output.append(y_)
    
print(classification_report(true_output,pred_output))
print(metrics.accuracy_score(true_output,pred_output))
    
print(confusion_matrix(true_output,pred_output))    

check=pd.DataFrame({'person_id':result,'count of depression':count,'true_count':true_count,'predicted label':pred_output,'true_label':true_output})

check.to_csv('/home/vboxuser/VoiceFiles/count_train_predicted.csv')

check

df_pred=pd.DataFrame({'person_id':result,'count of depression':count, 'percentage of depression':out_per,'label':pred_output,'true_label':true_output})

df_pred.tail() 

df_pred.to_csv('/home/vboxuser/VoiceFiles/count_predicted.csv')

import seaborn as sns

sns.scatterplot(x='percentage of depression',y='true_label',data=df_pred,s=60,hue='true_label',palette=['green','red'])

dp=[]
for i in range (0,len(out_per)):
    if out_per[i]>9:
        dp.append(out_per[i])

len(dp) #includes true negative + false negative

df_pred.sort_values(by=['label'])

##################################################################

SVM_model = SVC(kernel='rbf',C=10, gamma=1, class_weight='balanced')

SVM_model.fit(X,Y)

true_per=[]
out_per=[]
true_output=[]
pred_output=[]

for i in range (0,len(result)):
    test=df_test[df_test['subject'].isin(result[i])] 
    X_test=test[test.columns.intersection(feature_list)]
    X_test=n.normalize_test(X_test)
    Y_test=test['Group']

    Y_predict_SVM=SVM_model.predict(X_test)
    
    per_pred=np.sum(Y_predict_SVM==1)/Y_predict_SVM.shape[0]*100
    if per_pred>6:
        y_=1
    else:
        y_=0
    
    y=mode(Y_test)
    per=np.sum(Y_test==1)/Y_test.shape[0]*100

    true_per.append(per)
    out_per.append(per_pred)
    true_output.append(y)
    pred_output.append(y_)

print(classification_report(true_output,pred_output))
print(metrics.accuracy_score(true_output,pred_output))
    
    

df_pred_SVM=pd.DataFrame({'percentage of depression':true_per,'label':pred_output,'true_label':true_output})

import seaborn as sns

sns.scatterplot(x='percentage of depression',y='true_label',data=df_pred_SVM,s=60,hue='true_label',palette=['green','red'])

#### 1.2 Obtaining a Classification Report:

#from sklearn.metrics import classification_report

#print(classification_report(Y_test,Y_predict_RF))

#### 2. Decision Tree Classifier:

#DF_model=DecisionTreeClassifier(max_depth=7,random_state=42,class_weight='balanced')

#DF_model.fit(X,Y)

#### 2.1 Predicting the output on test data:

#predicting on test data:
#Y_predict_DF=DF_model.predict(X_test)

#viewing accuracy score:
#print("The DF_model accuracy is given as : ",metrics.accuracy_score(Y_test,Y_predict_DF))

#### 2.2 Obtaining a Classification Report:

#print(classification_report(Y_test,Y_predict_DF))

#### 3. Logistic Regression Model:

#LR_model=LogisticRegression(class_weight='balanced')

#LR_model.fit(X,Y)

#### 3.1 Predicting the output on test data:

#predicting on test data:
#Y_predict_LR=LR_model.predict(X_test)

#### 3.2 Obtaining a Classification Report:

#viewing accuracy score:
#print("The LR_model accuracy is given as : ",metrics.accuracy_score(Y_test,Y_predict_LR))

#print(classification_report(Y_test,Y_predict_LR))

#### 4. Support Vector Machine Model:

#SVM_model.fit(X,Y)

#### 4.1 Predicting the output on test data:

#predicting on test data:
#Y_predict_SVM=SVM_model.predict(X_test)

#viewing accuracy score:
#print("The SVM_accuracy is given as : ",metrics.accuracy_score(Y_test,Y_predict_SVM))

#### 4.2 Obtaining a Classification Report:

#print(classification_report(Y_test,Y_predict_SVM))

<font color=red>**C=10 & gamma=1 with resampled gives the best results**</font>

### Feature Combination:

#### PCA (on selected features):

from sklearn.decomposition import PCA

pca=PCA(n_components=10)

x_trainpca=pca.fit_transform(X)
x_testpca=pca.transform(X_test)

##convert principal components for each sample from numpy array to df:
pca_df=pd.DataFrame(x_trainpca,columns=["PCA1","PCA2","PCA3","PCA4","PCA5","PCA6","PCA7","PCA8","PCA9","PCA10"])
#pca_df["y"]=Y
pca_df.tail()

#evr provides the amount of info or variance each principal component holds after projecting the data to a lower dimensional space:
print("evr is {}".format(pca.explained_variance_ratio_))

__inference__: 
* principal component 1 holds 52.9%
* principal component 2 holds 24.3%
* principal component 3 holds 9.91%
* principal component 4 holds 5.33% 

adding them all up we get over 92.4%(>90). Hence  we decide to take 4 components.

class classification_models:
#Random Forest Classifier:
    def RF_model(self,X,Y,X1,Y1):
        self.X=X
        self.Y=Y
        self.X1=X1
        self.Y1=Y1
        RF_model=RandomForestClassifier(n_estimators=100,class_weight='balanced',random_state=4222)
        RF_model.fit(X,Y)
        Y_predict_RF=RF_model.predict(X1)
        acc= metrics.accuracy_score(Y1,Y_predict_RF)
        
        tn,fp,fn,tp=confusion_matrix(Y1,Y_predict_RF).ravel()
        P=tp/(tp+fp)
        R=tp/(tp+fn)
    #if acc>0.68:
        return acc,P,R

import itertools
flist=[]
for l in range(len(feature_list)):
    for subset in itertools.combinations(feature_list, l):
        flist.append(subset)

len(flist)

obj=classification_models()
a,p,r=obj.RF_model(X,Y,X_test,Y_test)

print(a,p,r)

i=0
score=[]
result=[]

for i in range(86000,len(flist)):
        
    
    Xf_train_resampled=df_train_upsampled[df_train_upsampled.columns.intersection(flist[i])]
    Xf=n.normalize_train(Xf_train_resampled)
    Xf_test=df_test[df_test.columns.intersection(flist[i])]
    Xf_test=n.normalize_test(Xf_test)
    obj=classification_models()
    a,p,r=obj.RF_model(Xf,Y,Xf_test,Y_test)
    print(i)
    if a>0.70:
        res=[a,p,r]
        score=[str((flist[i])),a,p,r]
    #if res is not None:
        print(i,flist[i])
        print(res)
        
        result.append(score)
            

res_df_pr=pd.DataFrame(result,columns=['Feature','Accuracy','Precision','Recall'])

res_df_pr

res_df_pr.to_csv("feature scores for prosody.csv",index=True)

df_train_upsampled.head()

In [185]:
from sklearn.decomposition import PCA

In [186]:
pca=PCA(n_components=7)

In [187]:
x_trainpca=pca.fit_transform(X)
x_testpca=pca.transform(X_test)

In [193]:
p=df_train['subject']

In [196]:
##convert principal components for each sample from numpy array to df:
pca_df=pd.DataFrame(x_trainpca,columns=["PCA1","PCA2","PCA3","PCA4","PCA5","PCA6","PCA7"])
pca_df.head()

,PCA1,PCA2,PCA3,PCA4,PCA5,PCA6,PCA7
0,-0.341421,3.714983,1.610782,0.257786,-1.658443,-1.421525,-3.075615
1,-8.471369,9.837467,1.491239,1.816696,-0.014503,-1.809848,0.541951
2,-5.993691,9.331194,2.695135,1.221735,-2.323967,1.166851,0.356857
3,6.210829,8.599817,3.383637,-1.326966,-2.583288,-1.606403,-1.034984
4,4.544794,5.133944,2.050825,0.725469,-1.101084,2.302023,-5.571868


In [189]:
print("evr is {}".format(pca.explained_variance_ratio_))

evr is [0.28305224 0.08053063 0.05866287 0.04674477 0.03333245 0.0268234
 0.02179495]


In [190]:
pca_df.to_csv('/home/vboxuser/VoiceFiles/articulation_pca.csv')